# Funciones utilizadas en el proyecto

- Extraer y transformar datos de un archivo .xml

- Extraer y transformar datos de una web

def crearDriver():
    opciones=Options()

    opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
    opciones.add_experimental_option('useAutomationExtension', False)
    opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
    opciones.add_argument('--start-maximized')         # comienza maximizado
    #opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
    #opciones.add_extension('driver_folder/adblock.crx')       # adblocker
    opciones.add_argument('--incognito')
    # opciones.add_argument('user-data-dir=cookies')    # mantiene las coockies

    driver = webdriver.Chrome(options = opciones)
    return driver


In [ ]:
def extraerRazas(driver, url: str)->list:
       driver.get(url)
    time.sleep(3)


    cookiesButton = driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
    if cookiesButton:
        cookiesButton.click()
        time.sleep(2)

    razas = []

    while True:
        gatos = driver.find_elements(By.CLASS_NAME, 'results-view-name')
        if not gatos:
            break
        razas = razas + [e.text for e in gatos] #extend

        next_page_button = [x for x in driver.find_element(By.CLASS_NAME, 'pager').find_elements(By.CSS_SELECTOR, 'a') if x.get_attribute('title')=='Go to next page']

        if not next_page_button:   # Si no lo encuentra, no hay más páginas
            break
        try:
            next_page_button[0].click()
        except:
            next_page_button[0].click()
        finally:
            time.sleep(3)

    driver.quit()
    return razas

- Extraer y transformar datos de un archivo .csv